In [1]:
from openml import tasks

import torch

import sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold

from benchmark import NeuralNetwork, Dataset

In [2]:
oml_task_diabetes = tasks.get_task(37)

In [3]:
X, y, categorical_indicator, attribute_names = oml_task_diabetes.get_dataset().get_data()

In [4]:
X

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,tested_positive
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,tested_negative
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,tested_positive
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,tested_negative
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,tested_positive
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,tested_negative
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,tested_negative
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,tested_negative
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,tested_positive


In [5]:
Xy = X.copy()

In [14]:
tmp = Dataset(
    X=Xy.loc[:, Xy.columns != 'class'],
    y=Xy.loc[:, 'class'],
    class_pos='tested_positive'
)
tmp[2]
len(tmp)

768

In [7]:
categorical_indicator

[False, False, False, False, False, False, False, False, True]

In [10]:
# both cf. https://pytorch.org/tutorials/beginner/introyt/trainingyt.html
def train(optimizer, loss_fn, model, epochs, dataset_train, dataset_test, batch_size):
    eval_loss = eval(loss_fn, model, dataset_test, batch_size)
    print(f'epoch 0 / {epochs}, eval loss {eval_loss}')

    loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=False)  # TODO: set shuffle to True

    for epoch in range(epochs):
        model.train()  # training mode, put here as we eval at the end of each epoch
        running_epoch_loss = 0

        for batch_input, batch_target in loader_train:  # divide data in mini batches
            optimizer.zero_grad()  # set gradients to 0
            batch_output = model(batch_input).flatten()

            batch_loss = loss_fn(batch_output, batch_target)
            running_epoch_loss += batch_loss.detach().item()
            batch_loss.backward()  # compute gradients

            optimizer.step()  # update weights
        running_epoch_loss /= len(loader_train)

        print(f'epoch {epoch + 1} / {epochs}, train loss {running_epoch_loss}')
        eval_loss = eval(loss_fn, model, dataset_test, batch_size)
        print(f'epoch {epoch + 1} / {epochs}, eval loss {eval_loss}')


def eval(loss_fn, model, dataset_test, batch_size, verbose=False):
    loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size)  # no need to shuffle in test
    
    model.eval()  # eval mode
    running_loss = 0

    for batch_input, batch_target in loader_test:
        with torch.no_grad():
            batch_output = model(batch_input).flatten()
            
        batch_loss = loss_fn(batch_output, batch_target)
        running_loss += batch_loss.detach().item()
    running_loss /= len(loader_test)

    if verbose:
        print(f'eval loss {running_loss}')
    return running_loss

In [ ]:
# outer
data_train_test, data_val = train_test_split(
    Xy,
    train_size=2/3,
    shuffle=True,
    stratify=Xy.loc[:, 'class']
)

# reset indices as StratifiedKFold assumes 0-(n-1) index
data_train_test = data_train_test.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)

# inner
cv_inner = StratifiedKFold(
    n_splits=5,
    shuffle=False  # TODO: set to True
)

for i, (indices_train, indices_test) in enumerate(cv_inner.split(X=data_train_test, y=data_train_test.loc[:, 'class'])):
    print(f'fold {i + 1} / {cv_inner.get_n_splits()}')

    data_train = data_train_test.loc[indices_train, :]
    dataset_train = Dataset(
        X=data_train.loc[:, data_train.columns != 'class'],
        y=data_train.loc[:, 'class'],
        class_pos='tested_positive'
    )

    data_test = data_train_test.loc[indices_test, :]
    dataset_test = Dataset(
        X=data_test.loc[:, data_test.columns != 'class'],
        y=data_test.loc[:, 'class'],
        class_pos='tested_positive'
    )

    '''
    in each inner fold, run EAGGA + find an optimal pareto front of configurations for this specific fold
    then in the outer fold (holdout), compare each fold-specific optimal pareto front + select optimum
    '''

    hp_configs = {
        'total_layers': [3],
        'nodes_per_hidden_layer': [3]
    }
    metrics_eval = {
        'auc': list(),
        'ni': list(),
        'nf': list(),
        'nnm': list()
    }
    epochs = 10
    batch_size = 8
    # HPO loop starts here
    while(True):  # TODO: define stopping criterion + remove break from end of loop
        model = NeuralNetwork(
            input_size=len(data_train_test.columns) - 1,
            output_size=1,  # we only use binary datasets
            total_layers=hp_configs['total_layers'][-1],
            nodes_per_hidden_layer=hp_configs['nodes_per_hidden_layer'][-1]
        )

        optimizer = torch.optim.AdamW(model.parameters())
        loss_fn = torch.nn.BCEWithLogitsLoss()
        train(optimizer, loss_fn, model, 5, dataset_train, dataset_test, batch_size)

        # TODO: evaluate performance + add to auc, NI, NF, NNM

        # TODO: EA on hp_total_layers + hp_nodes
        # TODO: implement group structure

        # until stopping criterion is met
        break

fold 1 / 5
<generator object Module.parameters at 0x00000202A18C3220>
epoch 0 / 5, eval loss 65.38461538461539
epoch 1 / 5, train loss 63.94230769230769
epoch 1 / 5, eval loss 65.38461538461539


c:\workplace\uni\ws-24-25\seminar\ws2425-tab-ml\code\.venv\Lib\site-packages\torch\nn\modules\module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epoch 2 / 5, train loss 63.94230769230769
epoch 2 / 5, eval loss 65.38461538461539
epoch 3 / 5, train loss 63.94230769230769
epoch 3 / 5, eval loss 65.38461538461539
epoch 4 / 5, train loss 63.94230769230769
epoch 4 / 5, eval loss 65.38461538461539
epoch 5 / 5, train loss 63.94230769230769
epoch 5 / 5, eval loss 65.38461538461539
fold 2 / 5
<generator object Module.parameters at 0x00000202A5D217E0>
epoch 0 / 5, eval loss 65.38461538461539


c:\workplace\uni\ws-24-25\seminar\ws2425-tab-ml\code\.venv\Lib\site-packages\torch\nn\modules\module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epoch 1 / 5, train loss 63.94230769230769
epoch 1 / 5, eval loss 65.38461538461539
epoch 2 / 5, train loss 63.94230769230769
epoch 2 / 5, eval loss 65.38461538461539
epoch 3 / 5, train loss 63.94230769230769
epoch 3 / 5, eval loss 65.38461538461539
epoch 4 / 5, train loss 63.94230769230769
epoch 4 / 5, eval loss 65.38461538461539
epoch 5 / 5, train loss 63.94230769230769
epoch 5 / 5, eval loss 65.38461538461539
fold 3 / 5
<generator object Module.parameters at 0x00000202A18C3760>
epoch 0 / 5, eval loss 66.34615384615384
epoch 1 / 5, train loss 64.66346153846153
epoch 1 / 5, eval loss 66.34615384615384


...

epoch 2 / 5, train loss 64.66346153846153
epoch 2 / 5, eval loss 66.34615384615384
epoch 3 / 5, train loss 64.66346153846153
epoch 3 / 5, eval loss 66.34615384615384
epoch 4 / 5, train loss 64.66346153846153
epoch 4 / 5, eval loss 66.34615384615384
epoch 5 / 5, train loss 64.66346153846153
epoch 5 / 5, eval loss 66.34615384615384
fold 4 / 5
<generator object Module.parameters at 0x00000202A18C3220>
epoch 0 / 5, eval loss 64.74358954796425


c:\workplace\uni\ws-24-25\seminar\ws2425-tab-ml\code\.venv\Lib\site-packages\torch\nn\modules\module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epoch 1 / 5, train loss 64.90384615384616
epoch 1 / 5, eval loss 64.74358954796425
epoch 2 / 5, train loss 64.90384615384616
epoch 2 / 5, eval loss 64.74358954796425
epoch 3 / 5, train loss 64.90384615384616
epoch 3 / 5, eval loss 64.74358954796425
epoch 4 / 5, train loss 64.90384615384616
epoch 4 / 5, eval loss 64.74358954796425
epoch 5 / 5, train loss 64.90384615384616
epoch 5 / 5, eval loss 64.74358954796425
fold 5 / 5
<generator object Module.parameters at 0x00000202A18C3BC0>
epoch 0 / 5, eval loss 65.06410275972806
epoch 1 / 5, train loss 65.625
epoch 1 / 5, eval loss 65.06410275972806


c:\workplace\uni\ws-24-25\seminar\ws2425-tab-ml\code\.venv\Lib\site-packages\torch\nn\modules\module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


epoch 2 / 5, train loss 65.625
epoch 2 / 5, eval loss 65.06410275972806
epoch 3 / 5, train loss 65.625
epoch 3 / 5, eval loss 65.06410275972806
epoch 4 / 5, train loss 65.625
epoch 4 / 5, eval loss 65.06410275972806
epoch 5 / 5, train loss 65.625
epoch 5 / 5, eval loss 65.06410275972806
